<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900"> **Extra Notebook:** Agent Executer Cell </font>

**This notebook is there for those confused about how the Agent Executor loop works but don't want to scrape through the source code too much.** This may not be completely up-to-date, but the code is simplified and the logic should be pretty transparent. 

Checking out the [AgentExecutor call method](https://github.com/langchain-ai/langchain/blob/c2d1d903fa35b91018b4d777db2b008fcbaa9fbc/langchain/agents/agent.py#L378), you'll notice that the event loop looks something like this: 

```python
class AgentExecutor(Chain, BaseModel):
    def _call(self, inputs: Dict[str, str]) -> Dict[str, Any]:

        # Do any preparation necessary when receiving a new input.
        self.agent.prepare_for_new_call()

        ## Setup the event loop
        for iterations in range(self.max_iterations):
            # Call the LLM to see what to do.
            output = self.agent.plan(intermediate_steps, **inputs)

            # If the agent produced an AgentFinish output, finish up.
            if isinstance(output, AgentFinish):
                return self._return(output, intermediate_steps)

            # Otherwise we lookup and try to use the tool
            if output.tool in name_to_tool_map:
                tool = name_to_tool_map[output.tool]
                observation = tool.func(output.tool_input) ## Get observation from the tool
                return_direct = tool.return_direct         ## Should we break out of the loop?
            else:
                observation = f"{output.tool} is not a valid tool, try another one."

            ## Record the recent interaction in the intermediate_steps
            intermediate_steps.append((output, observation))

            ## If our tool produced a final output, finish up.
            if return_direct:
                output = AgentFinish({self.agent.return_values[0]: observation}, "")
                return self._return(output, intermediate_steps)

        ## If a good output is never found, just give up and perform default exit
        output = self.agent.return_stopped_response(
            self.early_stopping_method, intermediate_steps, **inputs
        )
        return self._return(output, intermediate_steps)

```

This leads us into what kinds of logic the [ZeroShotAgent](https://github.com/langchain-ai/langchain/blob/c2d1d903fa35b91018b4d777db2b008fcbaa9fbc/langchain/agents/mrkl/base.py#L51) contains, which is summarized here:

```python
class ZeroShotAgent(Agent):
    
    ##############################################################################
    ## From Agent superclass
    llm_chain: LLMChain
    allowed_tools: Optional[List[str]] = None
    return_values: List[str] = ["output"]
    finish_tool_name: str = "Final Answer"
    
    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do"""
        full_inputs = self.get_full_inputs(intermediate_steps, **kwargs)
        action = self._get_next_action(full_inputs)
        if action.tool == self.finish_tool_name:
            return AgentFinish({"output": action.tool_input}, action.log)
        return action
    
    def _get_next_action(self, full_inputs: Dict[str, str]) -> AgentAction:
        """Given the full input to the LLM, make the next prediction"""
        full_output = self.llm_chain.predict(**full_inputs)
        action, action_input = self._extract_tool_and_input(full_output)
        return AgentAction(
            tool=action, tool_input=action_input, log=full_output
        )

    ##############################################################################
    ## From ZeroShotAgent specifically
    def _extract_tool_and_input(self, text: str) -> Optional[Tuple[str, str]]:
        """
        Parse out the action and input from the LLM output.
        The string starting with "Action:" and the following string starting
        with "Action Input:" should be separated by a newline.
        """
        if "Final Answer:" in llm_output:
            return "Final Answer", llm_output.split("Final Answer:")[-1].strip()
        action, action_input = format_match(r"Action: (.*?)\nAction Input: (.*)", llm_output)
        if not action or not action_input: 
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        return action, action_input
```

## The Starter Code for the Assignment

```python
####################################################################################
## TODO: Your workspace is below
    
llama_full_prompt = PromptTemplate.from_template(
    template="<s>[INST]<<SYS>>{sys_msg}<</SYS>>\n\nContext:\n{history}\n\nHuman: {input}\n[/INST] {primer}",
)

llama_prompt = llama_full_prompt.partial(
    sys_msg = ( 
        "You are a helpful, respectful and honest AI assistant."
        "\nAlways answer as helpfully as possible, while being safe."
        "\nPlease be brief and efficient unless asked to elaborate, and follow the conversation flow."
        "\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content."
        "\nEnsure that your responses are socially unbiased and positive in nature."
        "\nIf a question does not make sense or is not factually coherent, explain why instead of answering something incorrect." 
        "\nIf you don't know the answer to a question, please don't share false information."
        "\nIf the user asks for a format to output, please follow it as closely as possible."
    ),
    primer = "",
    history = "",
)

####################################################################################
## THESE MIGHT BE USEFUL IMPORTS!

from langchain.chains import ConversationChain

img_pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
emo_pipe = pipeline('sentiment-analysis', 'SamLowe/roberta-base-go_emotions')  
zsc_pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
tox_pipe = pipeline("text-classification", model="nicholasKluge/ToxicityModel")
## WARNING: toxic_pipe returns the reward, where reward = 1 - toxicity

###################################################################################


class MyAgent(MyAgentBase):
    
    ## Instance methods that can be passed in as BaseModel arguments. 
    ## Will be associated with self
    
    general_prompt : PromptTemplate
    llm            : BaseLLM
    
    general_chain  : Optional[LLMChain]
    max_messages   : int                   = Field(10, gt=1)
    
    temperature    : float                 = Field(0.6, gt=0, le=1)
    max_new_tokens : int                   = Field(128, ge=1, le=2048)
    eos_token_id   : Union[int, List[int]] = Field(2, ge=0)
    gen_kw_keys = ['temperature', 'max_new_tokens', 'eos_token_id']
    gen_kw = {}
    
    user_toxicity  : float = 0.5
    user_emotion   : str = "Unknown"
    
    
    @root_validator
    def validate_input(cls, values: Any) -> Any:
        '''Think of this like the BaseModel's __init__ method'''
        if not values.get('general_chain'):
            llm = values.get('llm')
            prompt = values.get("general_prompt")
            values['general_chain'] = LLMChain(llm=llm, prompt=prompt)  ## <- Feature stop 
        values['gen_kw'] = {k:v for k,v in values.items() if k in values.get('gen_kw_keys')}
        return values
    

    def plan(self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any): 
        '''Takes in previous logic and generates the next action to take!'''
        
        ## [Base Case] Default message to start off the loop. TO NOT OVERRIDE
        tool, response = "Ask-For-Input Tool", "Hello World! How can I help you?"
        if len(intermediate_steps) == 0:
            return self.action(tool, response)
        
        ## History of past agent queries/observations
        queries      = [step[0].tool_input for step in intermediate_steps]
        observations = [step[1]            for step in intermediate_steps]
        last_obs     = observations[-1]    # Most recent observation (i.e. user input)

        #############################################################################
        ## FOR THIS METHOD, ONLY MODIFY THE ENCLOSED REGION
        
        ## [!] Probably a good spot for your user statistics tracking
        
        ## [Stop Case] If the conversation is getting too long, wrap it up
        if len(observations) >= self.max_messages:
            response = "Thanks so much for the chat, and hope to see ya later! Goodbye!"
            return self.action(tool, response, finish=True)
        
        ## [!] Probably a good spot for your input-augmentation steps

        ## [Default Case] If observation is provided and you want to respond... do it!
        with SetParams(llm, **self.gen_kw):
            response = self.general_chain.run(last_obs)
            
        ## [!] Probably a good spot for your output-postprocessing steps
        
        ## FOR THIS METHOD, ONLY MODIFY THE ENCLOSED REGION
        #############################################################################
        
        ## [Default Case] Send over the response back to the user and get their input!
        return self.action(tool, response)
    

    def reset(self):
        self.user_toxicity = 0
        self.user_emotion = "Unknown"
        if getattr(self.general_chain, 'memory', None) is not None:
            self.general_chain.memory.clear()  ## Hint about what general_chain should be...


####################################################################################
## Define how you want your conversation to go. You can also use your own input
## The below example in conversation_gen exercises some of the requirements.

student_name = "John Doe"   ## TODO: What's your name
ask_via_input = False       ## TODO: When you're happy, try supplying your own inputs

def conversation_gen():
    yield f"Hello! How's it going? My name is {student_name}! Nice to meet you!"
    yield "Please tell me a little about deep learning!"
    yield "What's my name?"                                  ## Memory buffer
    yield "I'm not feeling very good -_-. What should I do"  ## Emotion sensor
    yield "No, I'm done talking! Thanks so much!"            ## Conversation ender
    yield "Goodbye!"                                         ## Conversation ender x2
    raise KeyboardInterrupt()

conversation_instance = conversation_gen()
converser = lambda x: next(conversation_instance)

if ask_via_input:
    converser = input  ## Alternatively, supply your own inputs

agent_kw = dict(
    llm = llm,
    general_prompt = llama_prompt,
    max_new_tokens = 128,
    eos_token_id = [2]   
)

agent_ex = AgentExecutor.from_agent_and_tools(
    agent = MyAgent(**agent_kw),
    tools=[AskForInputTool(converser).get_tool()], 
    verbose=True
)

## NOTE: You might want to comment this out to make testing the autograder easier
try: agent_ex.run("")
except KeyboardInterrupt: print("KeyboardInterrupt")
```

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>